# Basis Test

In [1]:
# Import package
using LinearAlgebra, Plots, Test
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

In [2]:
begin
    mat = spin((1, "xx"), (1, "yy"))
    for L = 2:6
        b1 = ParityFlipBasis(L=L, p=+1, z=+1)
        b2 = ParityFlipBasis(L=L, p=+1, z=-1)
        b3 = ParityFlipBasis(L=L, p=-1, z=+1)
        b4 = ParityFlipBasis(L=L, p=-1, z=-1)
        
        Ea = trans_inv_operator(mat, 2, L) |> Hermitian |> eigvals
        E1 = trans_inv_operator(mat, 2, b1) |> Hermitian |> eigvals
        E2 = trans_inv_operator(mat, 2, b2) |> Hermitian |> eigvals
        E3 = trans_inv_operator(mat, 2, b3) |> Hermitian |> eigvals
        E4 = trans_inv_operator(mat, 2, b4) |> Hermitian |> eigvals
        E = sort(vcat(E1, E2, E3, E4))
        norm(Ea - E) > 1e-12 && "Check L=$L"
    end
end

In [4]:
size.([b1,b2,b3,b4],1) |> println 
[b.I for b in [b1,b2,b3,b4]]

[2, 1, 0, 1]


4-element Vector{Vector{Int64}}:
 [1, 2]
 [1]
 []
 [2]